# Score Analysis 

I made a score analysis of the students' reading examination. The reading questions consist of inference, direct and indirect, main ideas, and vocabulary. 
To do the test, I used Google form. As for the result, I downloaded the file in .csv. Here is it looks like (The students' name are covered):
<img src = 'reading1.png' width = '1500' height= '350'>

1. Open the .csv file
2. Dropped Timestamp and Score because the columns are not needed. Questions need to be checked one by one.

In [5]:
import pandas as pd
read = pd.read_csv('reading.csv')
read = read.drop(['Timestamp','Score'], axis = 1)


This is what the data looks like after the two columns dropped.
<img src = 'reading2.png' width = '700' height= '350'>

3. Other than Name: column, the column names consist of NUMBER and QUESTION of the test. 
Since I only need numbers, I change all column names into numbers, including Name: column.
So, the Name: column is changed into '0'.

In [6]:

read.columns = [str(i) for i in range(len(read.columns))]
read.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50'],
      dtype='object')

4. Every column consists of answers with LETTERS and the description of the annswer. 
    For further use, what I need are the letters.
    To achieve that:
        a. Group the columns for questions.
        b. Extract the letter only.
        c. Merge the 0 column (name) with columns 
        for questions that have been extracted.

In [7]:
import numpy as np

  
col_num = read.iloc[:,1:51]
read_1 = col_num.apply(lambda c: c.str.extract('\((.*?)\)', expand=False))
#read.col_num= read_1
read_clean = pd.concat([read['0'], read_1], axis = 1)


<img src = 'extract.png' width = '1500' height= '350'>

5. Then, I group the numbers into some categoeries as well as the       
    correct answer to those numbers.
        
        For instance, I need to find out how many correct numbers student get for inference 
        questions.

In [8]:
#group the lesson details
infer = read_clean[['2', '4', '19', '25', '26', '33', '36', '44']]
infer_ans = ['D', 'A', 'D', 'B', 'D', 'C', 'D', 'A']

direct_qs = read_clean[['6', '8', '10', '15', '17', '21', '24', '34', '37', '39', '41', '42', '47']]
direct_ans = ['C', 'B', 'D', 'C', 'A', 'A', 'A', 'B', 'C', 'D', 'B', 'B', 'C']
main_ideas = read_clean[['1', '11', '13', '20', '30', '40', '50']]
main_ideas_ans = ['C','B', 'D', 'B', 'D', 'D', 'C']

vocab = read_clean[['5', '7', '9', '12', '14', '16', '18', '22', '23', '27', '28', '31', '32', '35','38', '43', '45','46', '48']]
vocab_ans = ['B', 'D', 'A', 'B', 'A', 'C', 'B', 'D', 'A', 'A', 'D', 'B', 'A', 'A', 'B', 'C', 'D', 'B', 'C']


6. Compare students' answers with correct answers.
7. For every correct question students get, it will be summed and put into new columns.

In [9]:
infer_correct = infer == infer_ans
read_clean['infer_total'] = infer_correct.sum(axis = 1)

direct_qs_correct = direct_qs == direct_ans
read_clean['direct_qs'] = direct_qs_correct.sum(axis = 1)

main_ideas_correct = main_ideas == main_ideas_ans
read_clean['main_ideas'] = main_ideas_correct.sum(axis = 1)

vocab_correct = vocab == vocab_ans
read_clean['vocabs'] = vocab_correct.sum(axis = 1)

<img src = 'sums.png' width = '400' height= '400'>

8. I want to get the description from correct answers student get for 
    every category. 

    For example, Student 1 gets all correct answers for Inference questions. It means his/her 
    ability to answer inference question was excellent.
    
9. Because every category of question has differenct amount of numbers, 
    so I separate the functions to get the desciption. 

10. The description for every category is randomised.

In [10]:
import random

all_true_des = [' was excellent', ' was very good', 'scored well'] 
half_true = [' was decent', ' was good', ' was average', 'was so-so']
not_really = [' needed improvement', ' needed a lof of practice', ' needed practice more']

#infer_total: 6-8(all_true), 3-5 (half_true), 0-2(not_really)
def set_desc(row):
    if row["infer_total"] < 3:
        return random.choice(not_really)
    elif row["infer_total"] < 6:
        return random.choice(half_true)
    elif row["infer_total"] <= 8:
        return random.choice(all_true_des)

#direct: 13
def direct(row):
    if row['direct_qs'] <= 4:
        return random.choice(not_really)
    elif row['direct_qs'] >= 5 and row['direct_qs'] <= 9:
        return random.choice(half_true )
    elif row['direct_qs'] <= 13:
        return random.choice(all_true_des)

def main(row): #7 
    if row["main_ideas"] < 2:
        return random.choice(not_really)
    elif row["main_ideas"] <= 7:
        return random.choice(all_true_des)
    elif row["main_ideas"] < 5:
        return random.choice(half_true)

# vocab : 14-19(all_true), 7-13 (half_true), 0-6(not_really)

def vocab(row):
    if row['vocabs'] <= 6:
        return random.choice(not_really)
    elif row['vocabs'] >= 7 and row['vocabs'] <= 13:
        return random.choice(half_true )
    elif row['vocabs'] <= 19:
        return random.choice(all_true_des)

11. Put all the descriptions for every categories in new columns.

In [11]:
read_clean = read_clean.assign(infer_desc=read_clean.apply(set_desc, axis=1))
read_clean = read_clean.assign(direct_qs_desc=read_clean.apply(direct, axis=1))
read_clean = read_clean.assign(main_desc=read_clean.apply(main, axis=1))
read_clean = read_clean.assign(vocab_desc=read_clean.apply(vocab, axis=1))

<img src = 'description.png' width = '800' height= '400'>

12. Then loop through the whole data to get the analysis.

In [12]:
read_clean['reading'] = random.choices(['When it comes to reading, ', 'As for reading, ', 'For reading comprehension, '], weights=[1, 1,1], k=len(read_clean))

for (student, infer, direct, main, vocab) in zip(read_clean['0'], read_clean['infer_desc'], read_clean['direct_qs_desc'], read_clean['main_desc'],read_clean['vocab_desc'] ): 
    print(f'When it comes to reading, {student}, the abilities to answer inference questions {infer}, direct and indirect questions {direct}, main idea questions {main}, and vocabulary questions {vocab}')
    

When it comes to reading, M. Zakky Al Wafa, the abilities to answer inference questions  was good, direct and indirect questions was so-so, main idea questions  was excellent, and vocabulary questions was so-so
When it comes to reading, Putri Fadhillah, the abilities to answer inference questions scored well, direct and indirect questions was so-so, main idea questions  was excellent, and vocabulary questions  was excellent
When it comes to reading, Fadhil Bintang Aliansyah , the abilities to answer inference questions scored well, direct and indirect questions  was average, main idea questions scored well, and vocabulary questions  was decent
When it comes to reading, Dinda Amanda khumairah, the abilities to answer inference questions  was excellent, direct and indirect questions  was excellent, main idea questions scored well, and vocabulary questions  was very good
When it comes to reading, Davina Aqila Putri , the abilities to answer inference questions  was very good, direct and i

<img src = 'analysis.png' width = '1000' height= '500'>